In [1]:
import nibabel as nib
import numpy as np
import torchio as tio
import os

C:\Users\wijflo\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def tuple_product(*args):
    product = 1
    for element in args:
        product *= element
    return product

In [4]:
def resize_images_to_reference (source_path, destination_path, ref_image_path=None, modification_string="", inclusion_string=""):
    files = os.listdir(source_path)

    # Select files to process.
    nifti_files = [file for file in files if (file.endswith('.nii.gz')) & (inclusion_string in file)]
    nii_img_shapes = []

    # First get a list of image shapes.
    for file in nifti_files:
        file_path = os.path.join(source_path, file)
        nii_img = nib.load(file_path)
        nii_img_shapes.append(nii_img.shape)
    
    nifti_files_to_process = nifti_files.copy()

    # If no reference image is used, dynamically choose a reference among selected files by choosing the biggest image.
    if ref_image_path is None:
        index_of_max_product = max(range(len(nii_img_shapes)), key=lambda i: tuple_product(nii_img_shapes[i]))
        del nifti_files_to_process[index_of_max_product]
        
        reference_filename = nifti_files[index_of_max_product]
        reference_image = tio.ScalarImage(os.path.join(source_path, reference_filename))
        
    else:
        reference_filename = os.path.basename(ref_image_path)
        reference_image = tio.ScalarImage(ref_image_path)
    
    # Define the reference image name, adding the modification string if necessary.
    split_name = reference_filename.split("_")
    if modification_string != "":
        split_name.insert(-1, modification_string)
    new_reference_img_name = "_".join(split_name)

    reference_image.save(os.path.join(destination_path, new_reference_img_name))

    print("Processed reference image ", reference_filename)

    # Use tio.Resample to resize all images to match the shape of the reference image.
    for file in nifti_files_to_process:
        file_path = os.path.join(source_path, file)
        
        image_to_resize = tio.ScalarImage(file_path)
        normalized_image = tio.Resample(target=reference_image)(image_to_resize)
        
        split_name = file.split("_")
        if modification_string != "":
            split_name.insert(-1, modification_string)
        new_img_name = "_".join(split_name)

        normalized_image.save(os.path.join(destination_path, new_img_name))
        
        print("Processed image ", file)

In [12]:
reference_image.save(os.path.join("D:\\pythonscripts_ETIS", new_reference_img_name))

In [17]:
basedir = "D:\\ADS_Inference\\Original_ADS"
folder_list = [folder for folder in os.listdir(basedir) if (".ipynb" not in folder) ]

for folder in folder_list:
    folder_path = os.path.join(basedir, folder)
    resize_images_to_reference(folder_path, folder_path, "Normalized")

Processed reference image  2018-104_01-10087-D0MR_b0.nii.gz
Processed image  2018-104_01-10087-D0MR_DWI.nii.gz
Processed reference image  2018-104_01-10113-D0MR_b0.nii.gz
Processed image  2018-104_01-10113-D0MR_DWI.nii.gz
Processed reference image  2018-104_01-10114-D0MR_b0.nii.gz
Processed image  2018-104_01-10114-D0MR_DWI.nii.gz
Processed reference image  2018-104_01-10116-D0MR_b0.nii.gz
Processed image  2018-104_01-10116-D0MR_DWI.nii.gz
Processed reference image  2018-104_01-10117-D0MR_b0.nii.gz
Processed image  2018-104_01-10117-D0MR_DWI.nii.gz
Processed reference image  2018-104_01-10118-D0MR_b0.nii.gz
Processed image  2018-104_01-10118-D0MR_DWI.nii.gz


In [7]:
basedir = "D:\\pythonscripts_ETIS\\Intensity_Normalized_imgs"

resize_images_to_reference(basedir, "D:\\pythonscripts_ETIS\\Test_SWI_Norm", ref_image_path="D:\\pythonscripts_ETIS\\Reference_Image_SkullStripping\\Reference_DWI_ADS_Normalized.nii.gz")

Processed reference image  Reference_DWI_ADS_Normalized.nii.gz
Processed image  2018-104_01-10087-D0MR_12_SWI_Images_swi3d1r_Normalized.nii.gz
Processed image  2018-104_01-10087-D0MR_8_TOF_3D_NEW_wo_ARTEFACTS_fl3d1r_t50_Normalized.nii.gz
Processed image  2018-104_01-10113-D0MR_6_3D_TOF_LARGE_Normalized.nii.gz
Processed image  2018-104_01-10113-D0MR_9_Ax_T2_GRE_Normalized.nii.gz
Processed image  2018-104_01-10114-D0MR_6_3D_TOF_LARGE_Normalized.nii.gz
Processed image  2018-104_01-10114-D0MR_7_Ax_3D_SWAN+CARTO_Normalized.nii.gz
Processed image  2018-104_01-10116-D0MR_7_AXIAL_T2_EG_fl2d1_Normalized.nii.gz
Processed image  2018-104_01-10116-D0MR_8_TOF_3D_WILLIS_FIN_fl3d1r_t70_Normalized.nii.gz
Processed image  2018-104_01-10117-D0MR_6_3D_TOF_LARGE_Normalized.nii.gz
Processed image  2018-104_01-10117-D0MR_7_Ax_T2_GRE_Normalized.nii.gz
Processed image  2018-104_01-10118-D0MR_12_SWI_Images_swi3d1r_Normalized.nii.gz
Processed image  2018-104_01-10118-D0MR_8_TOF_3D_RAPIDE_fl3d1r_t40_Normalized.n

In [23]:
ref_img_raw = tio.ScalarImage(("D:\\ADS_Algorithm\\ADSv1.3\\data\\examples\\Subject01\\Subject01_DWI.nii.gz"))
ref_image_resized = tio.Resize((192,224,192))(ref_img_raw)
ref_image_resized.save("D:\\pythonscripts_ETIS\\Reference_Image_SkullStripping\\Reference_DWI_ADS.nii.gz")